In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder

# Get data

In [14]:
df = pd.read_csv('dataset.csv')
df.head()

,Word,Language
0,DELLO,Italian
1,STESSO,Italian
2,AUTORE,Italian
3,Ora,Italian
4,sempre,Italian


# Prepare data
One-Hot Encode it, so the language is categorical

In [19]:
# Extract the language column
language_labels = df['Language']

# Initialize OneHotEncoder
encoder = OneHotEncoder()

# Reshape the language labels to a 2D array
language_labels_reshaped = language_labels.values.reshape(-1, 1)

# Fit and transform the language labels
onehot_encoded = encoder.fit_transform(language_labels_reshaped)

# Convert one-hot encoded array into DataFrame with appropriate column names
onehot_df = pd.DataFrame(onehot_encoded, columns=['Language'])

# Concatenate the one-hot encoded DataFrame with the original DataFrame
df_encoded = pd.concat([df, onehot_df], axis=1)

print(df_encoded)

            Word Language       Language
0          DELLO  Italian    (0, 3)\t1.0
1         STESSO  Italian    (0, 3)\t1.0
2         AUTORE  Italian    (0, 3)\t1.0
3            Ora  Italian    (0, 3)\t1.0
4         sempre  Italian    (0, 3)\t1.0
...          ...      ...            ...
79097      halla  Spanish    (0, 5)\t1.0
79098   contacto  Spanish    (0, 5)\t1.0
79099      Estar  Spanish    (0, 5)\t1.0
79100  Comprobar  Spanish    (0, 5)\t1.0
79101  picotazos  Spanish    (0, 5)\t1.0

[79102 rows x 3 columns]
